<a href="https://colab.research.google.com/github/ianellisjones/usn/blob/main/Fleet_app_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install streamlit pandas folium streamlit-folium requests beautifulsoup4

In [8]:
%%writefile app.py
"""
US NAVY FLEET TRACKER (Official Sources)
Version: 4.1.0 (Variable Fix)

A Streamlit web application that tracks the real-time deployment status
of the entire US Navy Surface Fleet using official ship history logs.

Fixes v4.1:
- Fixed NameError: Replaced 'LOCATION_COORDS' with 'PARSE_DICT' in scraping logic.
- Ensures locations are correctly identified from the text log.
"""

import streamlit as st
import pandas as pd
import folium
from streamlit_folium import st_folium
import requests
from bs4 import BeautifulSoup
import re

# ==============================================================================
# 1. CONFIGURATION
# ==============================================================================

st.set_page_config(page_title="US Navy Fleet Tracker", page_icon="⚓", layout="wide")

USER_AGENT = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'

# --- DICTIONARY 1: PARSING (Look for these words) ---
PARSE_DICT = {
    # Movement Overrides (Mapped to Oceans)
    "Pacific Ocean": ["departed san diego", "departed pearl harbor", "pacific ocean", "pacific", "departed bremerton", "departed everett", "departed seattle", "north pacific ocean"],
    "Atlantic Ocean": ["departed norfolk", "departed mayport", "atlantic ocean", "atlantic", "departed little creek", "off the coast of panama city", "north atlantic ocean"],

    # Ports
    "San Diego": ["san diego", "north island", "camp pendleton", "nassco", "bae systems", "point loma", "a-175", "a-173", "seal beach"],
    "Norfolk": ["norfolk", "portsmouth", "virginia beach", "newport news", "little creek", "berth 6, pier 2"],
    "Bremerton": ["bremerton", "kitsap"],
    "Yokosuka": ["yokosuka"],
    "Pearl Harbor": ["pearl harbor"],
    "Mayport": ["mayport", "jacksonville"],
    "Everett": ["everett"],
    "Seattle": ["seattle", "vigor shipyard"],
    "Singapore": ["singapore", "changi"],
    "Bahrain": ["bahrain", "manama"],
    "Dubai": ["dubai", "jebel ali"],
    "Qatar": ["qatar", "emiri", "mesaieed"],
    "Busan": ["busan", "south korea"],
    "Guam": ["guam", "apra"],
    "Sasebo": ["sasebo", "juliet basin"],
    "Manila": ["manila", "subic", "philippines"],
    "Klang": ["malaysia", "klang"],
    "Brunei": ["brunei", "muara"],
    "Okinawa": ["okinawa"],
    "Pascagoula": ["pascagoula", "ingalls"],
    "Portland, OR": ["swan island", "portland", "oregon"],
    "Groton": ["groton"],
    "Kings Bay": ["kings bay"],
    "Maine": ["maine", "bar harbor"],
    "Mobile, AL": ["mobile", "austal"],
    "Pensacola": ["pensacola"],
    "Marinette": ["marinette"],
    "Philadelphia": ["philadelphia"],
    "Orange, TX": ["orange, texas", "orange, tx"],
    "Charleston, SC": ["charleston"],
    "Port Canaveral": ["port canaveral"],
    "Port Angeles": ["port angeles"],
    "Thailand": ["gulf of thailand", "pattaya"],
    "Australia": ["perth", "fremantle", "brisbane", "sydney"],
    "Rota": ["rota", "spain"],
    "Italy": ["italy", "genoa", "naples", "augusta bay", "gaeta"],
    "Malta": ["valletta", "malta"],
    "Saipan": ["saipan"],
    "Diego Garcia": ["diego garcia"],
    "Souda Bay": ["souda bay", "crete"],

    # Regions
    "South China Sea": ["south china sea", "spratly", "luzon"],
    "Western Pacific": ["western pacific", "westpac", "san bernardino", "philippine sea", "sea of japan"],
    "Persian Gulf": ["persian gulf", "arabian gulf"],
    "Arabian Sea": ["arabian sea"],
    "Red Sea": ["red sea", "bab el-mandeb"],
    "Gulf of Oman": ["gulf of oman"],
    "Gulf of Aden": ["gulf of aden"],
    "Mediterranean": ["mediterranean", "adriatic", "gibraltar", "med sea"],
    "Caribbean Sea": ["caribbean", "st. croix", "trinidad", "tobago", "puerto rico", "virgin islands", "st. thomas"],
    "North Sea": ["north sea", "norwegian sea", "oslo"],
    "Suez Canal": ["suez"],
    "Panama Canal": ["panama canal"],

    # Oceans
    "Indian Ocean": ["indian ocean"],
}

# --- DICTIONARY 2: MAPPING (Where to put the pin) ---
COORD_DICT = {
    "norfolk": [36.96, -76.30], "san diego": [32.68, -117.18], "mayport": [30.39, -81.42],
    "pearl harbor": [21.35, -157.97], "bremerton": [47.55, -122.64], "kitsap": [47.70, -122.70],
    "everett": [47.98, -122.22], "seattle": [47.60, -122.33], "port angeles": [48.12, -123.43],
    "portland, or": [45.56, -122.71], "groton": [41.39, -72.08], "kings bay": [30.80, -81.53],
    "pascagoula": [30.34, -88.56], "mobile, al": [30.69, -88.04], "pensacola": [30.35, -87.27],
    "panama city": [30.15, -85.75], "marinette": [45.10, -87.62], "orange, tx": [30.09, -93.73],
    "philadelphia": [39.89, -75.17], "charleston, sc": [32.85, -79.97], "port canaveral": [28.41, -80.60],
    "maine": [43.90, -69.80], "yokosuka": [35.29, 139.66], "yokohama": [35.44, 139.63],
    "sasebo": [33.16, 129.71], "busan": [35.10, 129.11], "guam": [13.44, 144.65], "saipan": [15.21, 145.72],
    "okinawa": [26.30, 127.80], "singapore": [1.30, 103.85], "klang": [3.00, 101.38], "brunei": [4.53, 114.72],
    "manila": [14.59, 120.97], "subic": [14.80, 120.27], "thailand": [12.60, 100.90], "bahrain": [26.22, 50.61],
    "dubai": [25.26, 55.30], "qatar": [25.35, 51.55], "duqm": [19.66, 57.70], "diego garcia": [-7.31, 72.41],
    "australia": [ -32.05, 115.74], "rota": [36.62, -6.35], "italy": [40.84, 14.25], "gaeta": [41.21, 13.58],
    "malta": [35.90, 14.51], "souda bay": [35.48, 24.07],
    "south china sea": [12.00, 114.00], "philippine sea": [20.00, 130.00], "western pacific": [15.00, 135.00],
    "sea of japan": [40.00, 135.00], "persian gulf": [27.00, 51.00], "arabian sea": [15.00, 65.00],
    "red sea": [20.00, 38.00], "gulf of oman": [24.00, 58.00], "gulf of aden": [12.00, 48.00],
    "mediterranean": [35.00, 18.00], "caribbean sea": [15.00, -75.00], "north sea": [56.00, 3.00],
    "strait of gibraltar": [35.95, -5.60], "suez canal": [30.60, 32.33], "panama canal": [9.10, -79.67],
    "bab el-mandeb": [12.58, 43.33], "atlantic ocean": [33.00, -60.00], "pacific ocean": [25.00, -150.00],
    "indian ocean": [-5.00, 80.00], "underway / unknown": [0.0, 0.0]
}

# --- FLEET URLS (Full List) ---
FLEET_URLS = [
    # CVN
    "http://uscarriers.net/cvn68history.htm", "http://uscarriers.net/cvn69history.htm",
    "http://uscarriers.net/cvn70history.htm", "http://uscarriers.net/cvn71history.htm",
    "http://uscarriers.net/cvn72history.htm", "http://uscarriers.net/cvn73history.htm",
    "http://uscarriers.net/cvn74history.htm", "http://uscarriers.net/cvn75history.htm",
    "http://uscarriers.net/cvn76history.htm", "http://uscarriers.net/cvn77history.htm",
    "http://uscarriers.net/cvn78history.htm",
    # Amphibs
    "http://uscarriers.net/lhd1history.htm", "http://uscarriers.net/lhd2history.htm",
    "http://uscarriers.net/lhd3history.htm", "http://uscarriers.net/lhd4history.htm",
    "http://uscarriers.net/lhd5history.htm", "http://uscarriers.net/lhd7history.htm",
    "http://uscarriers.net/lhd8history.htm", "http://uscarriers.net/lha6history.htm",
    "http://uscarriers.net/lha7history.htm",
    "http://uscarriers.net/lpd17history.htm", "http://uscarriers.net/lpd18history.htm",
    "http://uscarriers.net/lpd19history.htm", "http://uscarriers.net/lpd20history.htm",
    "http://uscarriers.net/lpd21history.htm", "http://uscarriers.net/lpd22history.htm",
    "http://uscarriers.net/lpd23history.htm", "http://uscarriers.net/lpd24history.htm",
    "http://uscarriers.net/lpd25history.htm", "http://uscarriers.net/lpd26history.htm",
    "http://uscarriers.net/lpd27history.htm", "http://uscarriers.net/lpd28history.htm",
    "http://uscarriers.net/lpd29history.htm",
    "http://uscarriers.net/lsd44history.htm", "http://uscarriers.net/lsd46history.htm",
    "http://uscarriers.net/lsd50history.htm", "http://uscarriers.net/lsd51history.htm",
    "http://uscarriers.net/lsd42history.htm", "http://uscarriers.net/lsd45history.htm",
    "http://uscarriers.net/lsd47history.htm", "http://uscarriers.net/lsd48history.htm",
    "http://uscarriers.net/lsd49history.htm", "http://uscarriers.net/lsd52history.htm",
    # Aux/Other
    "http://uscarriers.net/lcc19history.htm", "http://uscarriers.net/lcc20history.htm",
    "http://uscarriers.net/as39history.htm", "http://uscarriers.net/as40history.htm",
    "http://uscarriers.net/esb3history.htm", "http://uscarriers.net/esb4history.htm",
    "http://uscarriers.net/esb5history.htm", "http://uscarriers.net/esb6history.htm",
    "http://uscarriers.net/cg58history.htm", "http://uscarriers.net/cg60history.htm",
    "http://uscarriers.net/cg64history.htm", "http://uscarriers.net/cg67history.htm",
    "http://uscarriers.net/cg59history.htm", "http://uscarriers.net/cg65history.htm",
    "http://uscarriers.net/cg70history.htm", "http://uscarriers.net/cg71history.htm",
    "http://uscarriers.net/cg62history.htm",
    # LCS
    "http://uscarriers.net/lcs3history.htm", "http://uscarriers.net/lcs6history.htm",
    "http://uscarriers.net/lcs8history.htm", "http://uscarriers.net/lcs10history.htm",
    "http://uscarriers.net/lcs12history.htm", "http://uscarriers.net/lcs14history.htm",
    "http://uscarriers.net/lcs16history.htm", "http://uscarriers.net/lcs18history.htm",
    "http://uscarriers.net/lcs20history.htm", "http://uscarriers.net/lcs22history.htm",
    "http://uscarriers.net/lcs24history.htm", "http://uscarriers.net/lcs26history.htm",
    "http://uscarriers.net/lcs28history.htm", "http://uscarriers.net/lcs30history.htm",
    "http://uscarriers.net/lcs32history.htm", "http://uscarriers.net/lcs34history.htm",
    "http://uscarriers.net/lcs36history.htm", "http://uscarriers.net/lcs13history.htm",
    "http://uscarriers.net/lcs15history.htm", "http://uscarriers.net/lcs17history.htm",
    "http://uscarriers.net/lcs19history.htm", "http://uscarriers.net/lcs21history.htm",
    "http://uscarriers.net/lcs23history.htm", "http://uscarriers.net/lcs25history.htm",
    "http://uscarriers.net/lcs27history.htm", "http://uscarriers.net/lcs29history.htm",
    # DDG
    "http://uscarriers.net/ddg51history.htm", "http://uscarriers.net/ddg52history.htm",
    "http://uscarriers.net/ddg53history.htm", "http://uscarriers.net/ddg54history.htm",
    "http://uscarriers.net/ddg55history.htm", "http://uscarriers.net/ddg56history.htm",
    "http://uscarriers.net/ddg57history.htm", "http://uscarriers.net/ddg58history.htm",
    "http://uscarriers.net/ddg59history.htm", "http://uscarriers.net/ddg60history.htm",
    "http://uscarriers.net/ddg61history.htm", "http://uscarriers.net/ddg62history.htm",
    "http://uscarriers.net/ddg63history.htm", "http://uscarriers.net/ddg64history.htm",
    "http://uscarriers.net/ddg65history.htm", "http://uscarriers.net/ddg66history.htm",
    "http://uscarriers.net/ddg67history.htm", "http://uscarriers.net/ddg68history.htm",
    "http://uscarriers.net/ddg69history.htm", "http://uscarriers.net/ddg70history.htm",
    "http://uscarriers.net/ddg71history.htm", "http://uscarriers.net/ddg72history.htm",
    "http://uscarriers.net/ddg73history.htm", "http://uscarriers.net/ddg74history.htm",
    "http://uscarriers.net/ddg75history.htm", "http://uscarriers.net/ddg76history.htm",
    "http://uscarriers.net/ddg77history.htm", "http://uscarriers.net/ddg78history.htm",
    "http://uscarriers.net/ddg79history.htm", "http://uscarriers.net/ddg80history.htm",
    "http://uscarriers.net/ddg81history.htm", "http://uscarriers.net/ddg82history.htm",
    "http://uscarriers.net/ddg83history.htm", "http://uscarriers.net/ddg84history.htm",
    "http://uscarriers.net/ddg85history.htm", "http://uscarriers.net/ddg86history.htm",
    "http://uscarriers.net/ddg87history.htm", "http://uscarriers.net/ddg88history.htm",
    "http://uscarriers.net/ddg89history.htm", "http://uscarriers.net/ddg90history.htm",
    "http://uscarriers.net/ddg91history.htm", "http://uscarriers.net/ddg92history.htm",
    "http://uscarriers.net/ddg93history.htm", "http://uscarriers.net/ddg94history.htm",
    "http://uscarriers.net/ddg95history.htm", "http://uscarriers.net/ddg96history.htm",
    "http://uscarriers.net/ddg97history.htm", "http://uscarriers.net/ddg98history.htm",
    "http://uscarriers.net/ddg99history.htm", "http://uscarriers.net/ddg100history.htm",
    "http://uscarriers.net/ddg101history.htm", "http://uscarriers.net/ddg102history.htm",
    "http://uscarriers.net/ddg103history.htm", "http://uscarriers.net/ddg104history.htm",
    "http://uscarriers.net/ddg105history.htm", "http://uscarriers.net/ddg106history.htm",
    "http://uscarriers.net/ddg107history.htm", "http://uscarriers.net/ddg108history.htm",
    "http://uscarriers.net/ddg109history.htm", "http://uscarriers.net/ddg110history.htm",
    "http://uscarriers.net/ddg111history.htm", "http://uscarriers.net/ddg112history.htm",
    "http://uscarriers.net/ddg113history.htm", "http://uscarriers.net/ddg114history.htm",
    "http://uscarriers.net/ddg115history.htm", "http://uscarriers.net/ddg116history.htm",
    "http://uscarriers.net/ddg117history.htm", "http://uscarriers.net/ddg118history.htm",
    "http://uscarriers.net/ddg119history.htm", "http://uscarriers.net/ddg120history.htm",
    "http://uscarriers.net/ddg121history.htm", "http://uscarriers.net/ddg122history.htm",
    "http://uscarriers.net/ddg123history.htm", "http://uscarriers.net/ddg125history.htm",
    "http://uscarriers.net/ddg1000history.htm", "http://uscarriers.net/ddg1001history.htm",
    "http://uscarriers.net/ddg1002history.htm",
]

# ==============================================================================
# 2. LOGIC ENGINE
# ==============================================================================

@st.cache_data(ttl=3600)
def scrape_fleet_data():
    fleet_data = []
    progress_bar = st.progress(0)
    status_text = st.empty()

    total_urls = len(FLEET_URLS)

    for i, url in enumerate(FLEET_URLS):
        try:
            progress_bar.progress((i + 1) / total_urls)

            hull_match = re.search(r'((?:cvn|lha|lhd|lpd|lsd|lcc|cg|lcs|as|esb|ddg)\d+)', url, re.IGNORECASE)
            hull = hull_match.group(1).upper() if hull_match else "UNK"
            status_text.text(f"Scanning: {hull}")

            response = requests.get(url, headers={'User-Agent': USER_AGENT}, timeout=10)
            soup = BeautifulSoup(response.content, 'html.parser')

            full_text = soup.get_text(separator='\n')
            lines = [line.strip() for line in full_text.split('\n') if line.strip()]
            clean_text = '\n'.join(lines)
            text_block = clean_text[-50000:] if len(clean_text) > 50000 else clean_text

            current_year = "Unknown"
            years = re.findall(r'(202[3-6])', text_block)
            if years: current_year = years[-1]

            status_entry = "No recent status found."
            status_date = current_year

            lines_block = text_block.split('\n')
            for line in reversed(lines_block):
                if any(y in line for y in ["2024", "2025", "2026"]) and any(k in line.lower() for k in ["moored", "underway", "arrived", "departed", "transited", "operations", "undocked"]):
                    status_entry = line
                    date_match = re.search(r'(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*\.?\s+\d{1,2}', line, re.IGNORECASE)
                    if date_match: status_date = date_match.group(0)
                    break

            # --- CATEGORIZE LOCATION (FIXED LOGIC) ---
            text_lower = status_entry.lower()
            loc_tag = "Underway / Unknown"

            # Movement Overrides
            if "departed san diego" in text_lower: loc_tag = "Pacific Ocean"
            elif "departed norfolk" in text_lower: loc_tag = "Atlantic Ocean"
            elif "departed pearl harbor" in text_lower: loc_tag = "Pacific Ocean"
            elif "departed mayport" in text_lower: loc_tag = "Atlantic Ocean"
            elif "departed little creek" in text_lower: loc_tag = "Atlantic Ocean"
            else:
                matches = []
                # Iterate over PARSE_DICT (Keywords), NOT COORD_DICT (Lat/Lon)
                for label, keywords in PARSE_DICT.items():
                    for k in list(keywords) if isinstance(keywords, list) else [keywords]:
                        idx = text_lower.rfind(str(k))
                        if idx != -1:
                            matches.append((idx, len(str(k)), label))

                if matches:
                    matches.sort(key=lambda x: (x[0], x[1]), reverse=True)
                    loc_tag = matches[0][2]

            # Determine Type
            if "CVN" in hull: s_type = "Carrier"
            elif any(x in hull for x in ["LHA", "LHD", "LPD", "LSD"]): s_type = "Amphib"
            elif "DDG" in hull: s_type = "Destroyer"
            elif "CG" in hull: s_type = "Cruiser"
            elif "LCS" in hull: s_type = "LCS"
            elif "LCC" in hull: s_type = "Command"
            elif "ESB" in hull: s_type = "Sea Base"
            elif "AS" in hull: s_type = "Tender"
            else: s_type = "Other"

            fleet_data.append({
                "Hull": hull,
                "Type": s_type,
                "Location": loc_tag.title(),
                "Date": status_date,
                "Status": status_entry
            })

        except Exception as e:
            continue

    progress_bar.empty()
    status_text.empty()
    return pd.DataFrame(fleet_data)

# ==============================================================================
# 3. MAPPING LOGIC
# ==============================================================================

def create_map(df):
    m = folium.Map(location=[20, 0], zoom_start=2, tiles="CartoDB positron")

    grouped = df.groupby("Location")

    for loc, ships in grouped:
        # USE COORD_DICT to plot the pin
        coords = COORD_DICT.get(loc.lower(), [0.0, 0.0])

        # Determine Icon
        if any(ships['Type'] == 'Carrier'): color, icon = 'blue', 'plane'
        elif any(ships['Type'] == 'Amphib'): color, icon = 'red', 'anchor'
        elif any(ships['Type'] == 'Command'): color, icon = 'purple', 'flag'
        elif any(ships['Type'] == 'Destroyer'): color, icon = 'black', 'rocket'
        elif any(ships['Type'] == 'Cruiser'): color, icon = 'darkgreen', 'crosshairs'
        elif any(ships['Type'] == 'Sea Base'): color, icon = 'cadetblue', 'ship'
        elif any(ships['Type'] == 'Tender'): color, icon = 'gray', 'wrench'
        else: color, icon = 'orange', 'ship'

        if coords == [0.0, 0.0]:
            color, icon, loc_label = "gray", "question", "LOCATION UNKNOWN"
        else:
            loc_label = loc

        # Sort ships inside popup
        type_order = {"Carrier":1, "Command":2, "Amphib":3, "Destroyer":4, "Cruiser":5, "Sea Base":6, "Tender":7, "LCS":8}
        ships = ships.sort_values(by="Type", key=lambda x: x.map(type_order).fillna(99))

        popup_html = f"""
        <div style="font-family:sans-serif; width:320px; max-height:300px; overflow-y:auto;">
            <h5 style="margin-bottom:5px; border-bottom:2px solid {color}">{loc_label} ({len(ships)})</h5>
            <table style="width:100%; font-size:11px;">
        """

        for _, row in ships.iterrows():
            t_col = {"Carrier": "darkblue", "Amphib": "darkred", "Command": "purple", "Destroyer": "black", "Cruiser": "green"}.get(row['Type'], "gray")
            popup_html += f"""
            <tr style="border-bottom:1px solid #eee;">
                <td style="font-weight:bold; width:60px; color:{t_col};">{row['Hull']}</td>
                <td>{row['Status'][:100]}...</td>
            </tr>
            """
        popup_html += "</table></div>"

        folium.Marker(
            location=coords,
            popup=folium.Popup(popup_html, max_width=350),
            tooltip=f"{loc_label} ({len(ships)} Ships)",
            icon=folium.Icon(color=color, icon=icon, prefix='fa')
        ).add_to(m)

    return m

# ==============================================================================
# 4. MAIN APP
# ==============================================================================

def main():
    st.title("⚓ US Navy Fleet Tracker")
    st.markdown("### Global Deployment Status")
    st.caption("Data Source: USCarriers.net (Official Logs)")

    with st.sidebar:
        st.header("Controls")
        if st.button("🔄 Refresh Data"):
            st.cache_data.clear()
            st.experimental_rerun()
        st.info("Tracking ~170 Ships:\nCVN, LHA/D, LPD, LSD, CG, DDG, LCS, LCC, ESB, AS")

    with st.spinner("Scanning Fleet... (1-2 Mins)"):
        df = scrape_fleet_data()

    col1, col2 = st.columns([1, 3])
    with col1:
        all_types = sorted(df['Type'].unique().tolist())
        selected_types = st.multiselect("Filter Class", all_types, default=all_types)

    filtered_df = df[df['Type'].isin(selected_types)]

    m1, m2, m3, m4 = st.columns(4)
    m1.metric("Total Ships", len(filtered_df))
    m2.metric("Carriers", len(filtered_df[filtered_df['Type'] == 'Carrier']))
    m3.metric("Amphibs", len(filtered_df[filtered_df['Type'] == 'Amphib']))
    m4.metric("Destroyers", len(filtered_df[filtered_df['Type'] == 'Destroyer']))

    st.subheader("Live Map")
    m = create_map(filtered_df)
    st_folium(m, width=1400, height=600)

    st.subheader("Fleet Status Log")
    st.dataframe(
        filtered_df[['Hull', 'Type', 'Location', 'Date', 'Status']],
        use_container_width=True,
        hide_index=True
    )

if __name__ == "__main__":
    main()

Overwriting app.py


In [10]:
!curl ipv4.icanhazip.com

34.23.98.65


In [11]:
!pkill streamlit
!pkill localtunnel
print("RESTARTING SERVER...")
!streamlit run app.py &>/content/logs.txt &
!npx localtunnel --port 8501

RESTARTING SERVER...
⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦your url is: https://all-cats-care.loca.lt
^C
